In [3]:
def generate_5param_STC(datasheet_values):
    """
    
    This function will return the 5 parameters of a single diode model of the solar cell given datasheet values
    
    Input  : Datasheet Parameters: Voc, Isc, Vm, Im, Ncell at "STC"
             
    Output : Returns the single diode parameter tuple (Rs, Rsh, Isat, Iph, eta)
    
    
    """
    Voc, Isc, Vm, Im, Ncell = datasheet_values
    
    Temp = 25
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    #Initial Guess Calculation (eta, Rs, Rsh)
    etai = (2*Vm-Voc) / (Ncell*Vt*(np.log((Isc-Im)/Isc)+Im/(Isc-Im)))
    Rsi = Vm/Im - (2*Vm-Voc) / ((Isc - Im) * (np.log((Isc-Im)/Isc) + Im/(Isc-Im)))
    if (Rsi<0):
        Rsi = 0
        num = 1 + (1-Im/Isc)*(np.log(1-Im/Isc))/(1-Vm/Voc)
        den = Im/Vm + (Isc-Im)*np.log(1-Im/Isc)/(Voc-Vm)
        Rshi = num/den
    else :
        Rshi = np.sqrt(Rsi/((Isc/(etai*Ncell*Vt)*np.exp((Rsi*Isc - Voc)/(etai*Ncell*Vt)))))

    # Solving them to get these 3 params
    def para(X):
        eta, Rs, Rsh = X
        f = [Im - Isc + (Vm + Im*Rs - Isc*Rs)/Rsh + (Isc - (Voc - Isc)/Rsh)*np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)), Im + Vm*((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Vm+Im*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Vm+Im*Rs)/(eta*Ncell*Vt)))+Rs/Rsh)), 1/Rsh + ((-((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*(np.exp((-Voc + Isc*Rs)/(eta*Ncell*Vt)))-(1/Rsh))/(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh)*Rs*np.exp((-Voc+Isc*Rs)/(eta*Ncell*Vt)))+Rs/Rsh))]
        return f

    param_sol = root(para,[etai,Rsi,Rshi])
    eta = param_sol.x[0]
    Rs = param_sol.x[1]
    Rsh = param_sol.x[2]
    if (Rs<0):
        Rs=0
    # Getting the remaining 2 parameters
    Isat = (Isc - (Voc - Isc*Rs)/Rsh)*(np.exp(-Voc/(eta*Ncell*Vt)));
    Iph = Isat * np.exp(Voc / (eta*Ncell*Vt)) + (Voc/Rsh);

    parameters = (Rs, Rsh, Isat, Iph, eta)
    
    return (parameters)
#print (generate_5param_STC((45.9, 9.25, 37.2, 8.76, 72)))

In [4]:
import pandas as pd

def generate_IV(module_param, Temp, datapoints, Ncell, Voc=None, Isc=None):
    """
    This function will generate I-V data points given all the 5 diode parameters 
    
    Input  : 5 parameters: Rs, Rsh, Isat, Iph, eta
             Voc, Temperature(Celcius), N(Number of data points), Ncell
             
    Output : Returns a dictionary of I-V data (size = N) and metadata (data, metadata)
    
    This function will spit out I-V data points. 
    
    ** This auto generates Voc and Isc also if not provided **
    **Temperature is just taken to calculate Vt**
    
    """
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    Rs, Rsh, Isat, Iph, eta = module_param
    
    # If the Voc argument is not passed explicitly, this will calculate Voc also from the 5 parameters
    if not Voc:             # If Voc=None
        Voc_implicit_eqn = lambda voc: Iph - Isat*(np.exp((voc) / (eta*Ncell*Vt))-1) - (voc)/Rsh # i=0 @Voc
        Vocr = root(Voc_implicit_eqn,50)
        Voc = np.round(Vocr.x[0], 2)
        #print ('Auto generated Voc is :', Voc)
    
    # If the Isc argument is not passed explicitly, this will calculate Isc also from the 5 parameters
    if not Isc:             # If Voc=None
        Isc_implicit_eqn = lambda isc: -isc + Iph - Isat*(np.exp((isc*Rs) / (eta*Ncell*Vt))-1) - (isc*Rs)/Rsh # v=0 @Isc
        Isc = np.round(root(Isc_implicit_eqn,50).x[0], 2)
        #print ('Auto generated Isc is :', Isc)    
    
    # Now solving the implicit equation of the solar cell 
    I_implicit_equn = lambda i: -i + Iph - Isat*(np.exp((v+i*Rs) / (eta*Ncell*Vt))-1) - (v+i*Rs)/Rsh
    V = np.linspace(0,Voc,datapoints)  #Generating the Voltage vector
    I = []
    for v in V:                        # For each value of Voltage, current is calculated
        ior = root(I_implicit_equn,10)
        io = ior.x[0]
        I.append(io)
    
    I = np.round(I, 2)                 # Rounding upto 2 places
    V = np.round(V, 2)
    
    ## Now calculating Pmax, FF, Im, Vm
    df = pd.DataFrame(dict(V=V, I=I))
    df['P'] = df['I']*df['V']
    values = df.loc[df['P'].idxmax()]
    Vm, Im, Pmax = values
    FF = np.round(Pmax/(Voc*Isc),2)
    
    #metadata = dict(Voc=Voc, Isc=Isc, Pmax=np.round(Pmax,2), Vmp=Vm, Imp=Im, FF=FF) 
    metadata = dict(Voc=[Voc], Isc=[Isc], Pmax=[np.round(Pmax,2)], Vmp=[Vm], Imp=[Im], FF=[FF]) 
    data = dict(x=V, y=I)
    
    return (data, metadata)              # Returning a dictionary of data and metadata

In [8]:
from bokeh.models import Slider, CustomJS
from bokeh.models.widgets import Button, TextInput, DataTable, TableColumn
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import Figure, output_notebook, show, ColumnDataSource, output_file
import pandas as pd
from scipy.optimize import root
import numpy as np
from os.path import dirname, join

output_notebook()

def plot_iv(doc):
    
    datasheet_values = (45.9, 9.25, 37.2, 8.76, 72)        # datasheet_values = Voc, Isc, Vm, Im, Ncell
    module_param = generate_5param_STC(datasheet_values)   # parameters = (Rs, Rsh, Isat, Iph, eta)
    
    data_pair = generate_IV(module_param, 25, 100, 72)
    source = ColumnDataSource(data=data_pair[0])
    
    module_param_name = ['Rs', 'Rsh', 'Isat', 'Iph', 'Eta']
    module_param      = [['{:0.2e}'.format(i)] for i in module_param]
    metasource = ColumnDataSource(data=dict(zip(module_param_name, module_param)))
    col = [
        TableColumn(field="Iph", title="Iph"),
        TableColumn(field="Isat", title="Isat"),
        TableColumn(field="Eta", title="Eta"),
        TableColumn(field="Rs", title="Rs"),
        TableColumn(field="Rsh", title="Rsh"),
    ]
    meta_table = DataTable(source=metasource, columns=col, height=80)
    
    
    plot = Figure(y_range = (-1, 10), x_range = (0,60))
    plot.xaxis.axis_label = 'Voltage (V)'
    plot.yaxis.axis_label = 'Current (I)'
    plot.scatter('x', 'y', source=source, line_width=3, line_alpha=0.6)
    
    Isc_input = TextInput(value='9.25', title='I_sc at STC')
    Im_input = TextInput(value='8.76', title='I_m')
    Voc_input = TextInput(value='45.9', title='V_oc')
    Vm_input = TextInput(value='37.2', title='V_m')
    Ncell_input = TextInput(value='72', title='No. of cells')
    datasize_input = TextInput(value='100', title='Data Size')
    submit = Button(label='Submit', button_type='success')
    dwnld = Button(label='Download (csv)',)
    
    def get_inputs():
        return ((float(Voc_input.value) ,float(Isc_input.value) ,float(Vm_input.value) ,float(Im_input.value) ,int(Ncell_input.value)),int(datasize_input.value))
    
    def update_plot():
        datasheet_values, size = get_inputs()
        module_param = generate_5param_STC(datasheet_values)
        data_pair = generate_IV(module_param, 25, size, datasheet_values[4])
        source.data = data_pair[0]    
        module_param_name = ['Rs', 'Rsh', 'Isat', 'Iph', 'Eta']
        module_param      = [['{:0.2e}'.format(i)] for i in module_param]
        metasource.data   = dict(zip(module_param_name, module_param))

    def download():
        df = pd.DataFrame(source.data)
        df.to_csv('iv-data.csv')
        
    submit.on_click(update_plot)
    dwnld.on_click(download)
    
    layout = row(plot, column(Isc_input, Voc_input, Vm_input, Im_input, Ncell_input, datasize_input, submit, dwnld),)
    doc.add_root(column(layout, meta_table))

show(plot_iv)

Loading BokehJS ...